Importing Libraries : 
- **random** library is used to sample random number for the key generation.
- **Crypto.Util** is used to sample a prime number for the OWP.

In [197]:
import random
from Crypto.Util import number


Global variables
- *length* is the length of maximum prime that can be sampled. It can be kept small for the code to run faster. 

In [268]:
p = 0 
length = 512
primelength = 0 
Key = None
gp = None

# Functions
- **start_session()** is used to sample the prime number for the OWP.
- **generator()** is used to get the generator for the group used in OWP
- **owp(str)** is used to send the OWP value corresponding to the input which is binary string format
- **innerproduct(str,str)** is used to calculate the value of inner product of the 2 binary strings
- **PRG_1(str)** is a PRG which increases the length of the output by 1 where the key is still a binary string.
- **PRG(str)** is a PRG which doubles the length of the output by 1 where the key is a binary string.
- **seed()** is a function which actually sets a suitable key as a binary string.
- **prf()** is PRF constructed using OWP. 

In [199]:
def start_session():
    global p
    global primelength
    while 1>0 :        
        p = number.getPrime(length)
        if(number.isPrime(2*p + 1,1e-10)):
             p = 2*p + 1
             primelength = len(bin(p)[2:])
             return

   

In [226]:
def generator():
    global p
    y = p - 1
    lstfact = []
    z = y // 2
    t  = (bin(z)[2:])
    
    generator = -1
    for j in range(2,p):
        
        if not ( (((j**(2)) % p)  == 1 ) ):

                lst = []
                i = len(t) - 1
                x = (j**(1)) % (p)
                while(i>=0):
                    if(t[i] == '1'):
                        lst.append(x)
                    x = (x*x) % (p)
                    i-=1
                ans = 1
                for yu in lst:
                    ans = (ans*(yu))%(p)
                if(ans!=1):
                    return j
                    # break

In [274]:
def owp(x):
    global p
    global gp
    # t  = (bin(x)[2:])
    lst = []
    i = len(x) - 1
    ans1 = (gp**(1)) % (p)
    while(i>=0):
        if(x[i] == '1'):
            lst.append(ans1)
        ans1 = (ans1*ans1) % (p)
        i-=1
    ans = 1
    for yu in lst:
        ans = (ans*(yu))%(p)
    return ans
    # return (gp**(int(x,2)))%p
    

In [283]:
def innerproduct(x,r):
    X = x
    R = r 
    result = None
    # print(len(X))
    for bit in range(len(X)):
        if result == None:
            result = int(X[bit])*int(R[bit])
        else :
            result = result ^ (int(X[bit])*int(R[bit]))
    return result

In [284]:
def prg_1(binKey): # str
    # goldreich -levin thm Idea
    # binKey = bin(key)[2:].rjust(length,'0')
    # print(binKey)
    # print(len(binKey))
    assert(len(binKey)%2 == 0)
    l = int(len(binKey)/2)
    x = binKey[:l]
    r = binKey[l:]
    prstring = bin(owp(x))[2:].rjust(int(primelength),'0') + r + str(innerproduct(x,r))
    # print(prstring)
    # print(len(prstring))
    assert(len(prstring) == len(binKey) + 1)
    return prstring


In [285]:
def prg(key):
    # length doubling prg
    prstring = ''
    lastRStr = key # type = str
    for num in range(len(key)):
        binLast = prg_1(lastRStr) # type = str
        prstring = prstring + binLast[0]
        lastRStr = binLast[1:]
    prstring += lastRStr
    # print(len(key))
    # print(len(prstring))
    assert(len(prstring) == 2 * len(key))
    
    return prstring

In [286]:
def seed():
    global Key
    Key = bin(random.randint(1,p-1))[2:].rjust(primelength,'0') + bin(random.randint(1,p-1))[2:].rjust(primelength,'0')

In [287]:
def prf(x):
    global Key
    bin_x = bin(x)[2:].rjust(primelength,'0')
    CurrKey = Key
    for i in range(len(bin_x)):
        binrandnum = prg(CurrKey)
        if (bin_x[i] == '1'):
            CurrKey = binrandnum[int(len(binrandnum)/2):]
        else:
            CurrKey = binrandnum[:int(len(binrandnum)/2)]
    return int(CurrKey,2)
    

Session starting Code: run it once to start the session. 

In [288]:
start_session()

seed()
# print(p)
gp = generator()
print(gp)
print(p)

2
24341046848427499226737761932898400682402764504748808660021425001916779132520649984808564425751404823153543916952291296238946277700959052671511194174683539


In [290]:
prf(123467124)

315140703676162069354439937611622593853865732612813903020079066250352150622683214833940214820821338028303905895623003707495286220983854862599505559906799648312214584763729178395246681780985892294458504783928378381590134626793706919546085475606893177351679816543171203891553612105292629253507984009686376687354

5124 - 9 min 14.5s